### **Aykırı Değer Problemini Çözme**

In [22]:
##################### GEÇMİŞ TANIMLAMALAR ##############################

# Kütüphaneler

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
# pip install missingno
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler


# Pandas ayarları
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)


# Veri setleri
def load_application_train():
    data = pd.read_csv('application_train.csv')
    return data

def load():
    data = pd.read_csv('titanic.csv')
    return data


# Tanımlama
df_train = load_application_train()
df = load()


# Fonksiyonlar
# outlier_thresholds fonksiyonu
# Alt ve üst sınırlarının belirlemek için kullanılacak
def outlier_thresholds(dataframe, col_name, q1 = 0.25, q3 = 0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    
    #IQR
    interquartile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquartile_range
    low_limit = quartile1 - 1.5 * interquartile_range
    return low_limit, up_limit

# Aykırı değer var mı yok mu kontrol eden fonksiyonu tanımlayalım. (return: True/False)(bool)
def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

    # Bunu da kullanabiliriz.
    # if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].empty:
    #     return False
    # else:
    #     return True
        
# Efsane fonksiyon

def grab_col_names(dataframe, cat_th=10, car_th=20):
    """
    
    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    -------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe.
        cat_th: int, optinal
                Kategorik değişkenlerinin sayısının üst sınırı.
        car_th: int, optional
                Kategorik fakat kardinal değişkenler için sınıf eşik değeri.
    
    Returns
    -------
        cat_cols: list
                Kategorik değişken listesi.
        num_cols: list
                Numerik değişken listesi.
        cat_but_car: list
                Kategorik kardinal değişken listesi.
    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtype == 'O']
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtype != 'O']
                   
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                     dataframe[col].dtype == 'O']
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtype != 'O']
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f"cat_cols: {len(cat_cols)}")
    print(f"num_cols: {len(num_cols)}")
    print(f"cat_but_car: {len(cat_but_car)}")
    print("******************")
    print(f"num_but_cat: {len(num_but_cat)}")
    return cat_cols, num_cols, cat_but_car

# grab_outliers fonksiyonu (eğer değişkene ait aykırı değer sayısı 10 dan fazla ise ilk 5 değerini göster, az ise hepsini göster)
def grab_outliers(dataframe, col_name, index=False):
    low, up = outlier_thresholds(dataframe, col_name)

    if dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].shape[0] > 0:
        print(dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].head())
    else:
        print(dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))])

    if index:
        outlier_index = dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].index
        return outlier_index




#### Silme

In [23]:
low, up = outlier_thresholds(df,"Fare")

In [24]:
# Gözlem sayısına bakalım
df.shape

(891, 12)

In [25]:
# Aykırı olmayan gözlem sayılarına bakalım
df[~((df["Fare"] < low) | (df["Fare"] > up))].shape

(775, 12)

In [26]:
# Aykırı olmayan gözlemleri dönen fonksiyonları yazalım.
def remove_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    df_wıthout_outliers = dataframe[~((dataframe[col_name] < low_limit) | (dataframe[col_name] > up_limit))]
    return df_wıthout_outliers

In [27]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)
num_cols = [col for col in num_cols if col not in "PassengerId"]

df.shape

Observations: 891
Variables: 12
cat_cols: 6
num_cols: 3
cat_but_car: 3
******************
num_but_cat: 4


(891, 12)

In [28]:
# Numerik değişkenlerin aykırı değerlerini kaldırıp yeni
for col in num_cols:
    new_df = remove_outlier(df, col)

In [29]:
# Kaç değişiklik olduğunu görebiliriz.
df.shape[0] - new_df.shape[0]

116

#### Baskılama Yöntemi (re-assigment with thresholds)

In [30]:
# Fare değişkeni için üst ve alt sınırları belirliyoruz.
low, up = outlier_thresholds(df, "Fare")

In [31]:
# Aykırı değerleri 
df[((df["Fare"] < low) | (df["Fare"] > up))]["Fare"]

1      71.283
27    263.000
31    146.521
34     82.171
52     76.729
61     80.000
62     83.475
72     73.500
88    263.000
102    77.287
118   247.521
120    73.500
124    77.287
139    79.200
151    66.600
159    69.550
180    69.550
195   146.521
201    69.550
215   113.275
218    76.292
224    90.000
230    83.475
245    90.000
256    79.200
257    86.500
258   512.329
262    79.650
268   153.463
269   135.633
275    77.958
290    78.850
291    91.079
297   151.550
299   247.521
305   151.550
306   110.883
307   108.900
310    83.158
311   262.375
318   164.867
319   134.500
324    69.550
325   135.633
332   153.463
334   133.650
336    66.600
337   134.500
341   263.000
366    75.250
369    69.300
373   135.633
375    82.171
377   211.500
380   227.525
385    73.500
390   120.000
393   113.275
412    90.000
435   120.000
438   263.000
445    81.858
453    89.104
484    91.079
486    90.000
496    78.267
498   151.550
504    86.500
505   108.900
520    93.500
527   221.779
537   

In [32]:
# Loc ile aykırı değerler
df.loc[((df["Fare"] < low) | (df["Fare"] > up)), "Fare"]

1      71.283
27    263.000
31    146.521
34     82.171
52     76.729
61     80.000
62     83.475
72     73.500
88    263.000
102    77.287
118   247.521
120    73.500
124    77.287
139    79.200
151    66.600
159    69.550
180    69.550
195   146.521
201    69.550
215   113.275
218    76.292
224    90.000
230    83.475
245    90.000
256    79.200
257    86.500
258   512.329
262    79.650
268   153.463
269   135.633
275    77.958
290    78.850
291    91.079
297   151.550
299   247.521
305   151.550
306   110.883
307   108.900
310    83.158
311   262.375
318   164.867
319   134.500
324    69.550
325   135.633
332   153.463
334   133.650
336    66.600
337   134.500
341   263.000
366    75.250
369    69.300
373   135.633
375    82.171
377   211.500
380   227.525
385    73.500
390   120.000
393   113.275
412    90.000
435   120.000
438   263.000
445    81.858
453    89.104
484    91.079
486    90.000
496    78.267
498   151.550
504    86.500
505   108.900
520    93.500
527   221.779
537   

In [33]:
# Üst sınırın üzerinde olan değerleri üst değere baskılayalım.
df.loc[(df["Fare"] > up), "Fare"] = up

In [34]:
# Bakalim üst sınırın üzerinde değerler kaldı mı?
df.loc[(df["Fare"] > up), "Fare"] # Boş

Series([], Name: Fare, dtype: float64)

In [35]:
# Alt sınırın altında olan değerler var mı?
df.loc[(df["Fare"] < low), "Fare"]
# Yok, çünkü Fare değişkenin alt sınırı sıfırın altında Fare değişkeninin ölçek türü oran olduğu için sıfırın altında gözleme sahip değildir.
# Oran sayısal değişkeni: Başlangıç noktası sıfır kabul edilen sayısal değişkenlerin ölçek türü orandır.

# Eğer olsaydı alt sınırın altında olan değerleri alt değere baskılamak için:
# df.loc[(df["Fare"] < low), "Fare"] = low

Series([], Name: Fare, dtype: float64)

In [37]:
# Baskılama fonksiyonunu yazalım
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe,variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [40]:
df = load()

In [41]:
df.shape

(891, 12)

In [43]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)
num_cols = [col for col in num_cols if col not in "PassengerId"]

Observations: 891
Variables: 12
cat_cols: 6
num_cols: 3
cat_but_car: 3
******************
num_but_cat: 4


In [44]:
# Aykırı değerler var mı?
for col in num_cols:
    print(col, check_outlier(df, col))

Age True
Fare True


In [45]:
# Aykırı değerleri baskılayalım
for col in num_cols:
    replace_with_thresholds(df, col)

In [46]:
# Tekrar bakalım aykırı değerler var mı?
for col in num_cols:
    print(col, check_outlier(df, col))

Age False
Fare False
